<a href="https://colab.research.google.com/github/mihirsakaria/solving-maths-problems-in-hindi/blob/main/Topic_Modeling_Using_Corextopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install corextopic

In [2]:
import numpy as np
import scipy.sparse as ss
import matplotlib.pyplot as plt

import corextopic.corextopic as ct
import corextopic.vis_topic as vt # jupyter notebooks will complain matplotlib is being loaded twice

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def create_stopwords():
  stopwords_df = pd.read_csv('/content/drive/MyDrive/Maths Problem Solving BTP/Hindi_Stopwords.csv')
  temp_list = stopwords_df.values.tolist()
  stopwords_list = []
  for i in temp_list:
    stopword = i[0]
    stopwords_list.append(stopword)
  return stopwords_list

In [5]:
def wordTokenize(text):
  return text.split(" ")


In [6]:
#Tokenising a text, removing stopwords and cleaning it
def clean_text_initial(text):
    text = "".join(text.split("."))
    text = "".join(text.split("।"))
    text = "".join(text.split("?"))
    
    text = "".join(text.split("/"))
    text = " ".join(text.split())
    text = ' '.join([x.lower() for x in wordTokenize(text) if x.lower() not in unwanted_list])
    text = ''.join([i for i in text if not i.isdigit()])
    
    return text

In [7]:

def create_vocab(train):
  unique_words = set()
  for text in train:
    tokens = wordTokenize(text)
    for token in tokens:
      unique_words.add(token)
  vocab = {}
  for index, word in enumerate(list(unique_words)):
    vocab[word] = index
  return vocab


In [8]:
def Vectorization(train):
  vocab = create_vocab(train)
  # print(vocab)
  row, col, val = [], [], []
  for idx, sentence in enumerate(train):
    count_word = {}
    tokens = wordTokenize(sentence)
    for token in tokens:
      count_word[token] = sentence.count(token)
    for word, count in count_word.items():
      col_index = vocab.get(word)
      if col_index >=0:
        row.append(idx)
        col.append(col_index)
        val.append(count)
  return csr_matrix((val, (row, col)), shape = (len(train), len(vocab)))


In [9]:

df1=pd.read_csv("/content/drive/MyDrive/Maths Problem Solving BTP/Hindi_Datasets.csv")

train = df1['Problem']
unwanted_list = create_stopwords()
train = train.apply(clean_text_initial)

words = create_vocab(train)
Words = list(words.keys())

vectors = Vectorization(train)
print(vectors.shape)
print(len(Words))

(446, 1741)
1741


In [10]:
anchor_words = [['संख्या' ,'प्रायिकता','संभावना','बांटा','यादृच्छिक','संभावित','संयोजनों','क्रम']]
# anchor_words2 = [['संख्या' ,'प्रायिकता','संभावना','बांटा','यादृच्छिक','संभावित','संयोजनों','क्रम','कितने','गणना','अपेक्षित','तरीकों','मूल्य']]
anchored_topic_model = ct.Corex(n_hidden=2, seed=2)
anchored_topic_model.fit(vectors, words = Words, anchors=anchor_words, anchor_strength=6);

for n in range(len(anchor_words)):
    topic_words,_,_ = zip(*anchored_topic_model.get_topics(topic=n))
    print('{}: '.format(n) + ', '.join(topic_words))

0: प्रायिकता, संख्या, यादृच्छिक, संभावना, क्या, रूप, व्यक्त, उभयनिष्ठ, उत्तर, भिन्न


In [11]:
result = anchored_topic_model.predict(vectors)
print(type(result))

<class 'numpy.ndarray'>


In [12]:
arithmatic = 0
prob = 0
false = 0
for i in range(len(result)):
  if i<215:
    if result[i][1] == True:
      arithmatic +=1
    if result[i][0] == True:
      false +=1
  else:
    if result[i][0] == True:
      prob +=1
    if(result[i][1] == True):
      false+=1
print(arithmatic)
print(prob)
print(false)
print(len(result))


100
123
81
446
